# 04. MNIST Full Experiments

This notebook runs the complete MNIST experiment suite for the research paper.

**Experiment Matrix:**
- Attacks: none, label_flip, backdoor, model_replacement
- Defenses: none, krum, trimmed_mean, fltrust, dp_sgd
- Distributions: IID, Non-IID (α=0.5), Non-IID (α=0.1)
- Seeds: 5 (for statistical significance)

**Estimated Time:** 4-6 hours on GPU

## Setup

In [ ]:
import sys
import os
import time
import json
from datetime import datetime

# Navigate to project root
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')

PROJECT_ROOT = os.getcwd()
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
import torch
import numpy as np

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")
if DEVICE == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
from experiments.run_experiments import ExperimentConfig, ExperimentRunner

---
## Configuration

In [ ]:
# MNIST Configuration
MNIST_CONFIG = {
    "dataset": "mnist",
    "num_clients": 10,
    "num_rounds": 10,
    "local_epochs": 1,
    "batch_size": 32,
    "learning_rate": 0.01,
}

# Seeds for reproducibility
SEEDS = [42, 123, 456, 789, 1024]  # 5 seeds for paper-quality results
# SEEDS = [42]  # Use single seed for quick testing

# Distributions
DISTRIBUTIONS = [
    ("iid", 1.0),
    ("noniid", 0.5),
    ("noniid", 0.1),
]

# Attack configurations
ATTACKS = {
    "none": {"enabled": False},
    "label_flip": {
        "enabled": True,
        "type": "label_flip",
        "malicious_clients": [0, 1],  # 20% malicious
        "poison_ratio": 0.3,
        "target_class": 8,
    },
    "backdoor": {
        "enabled": True,
        "type": "backdoor",
        "malicious_clients": [0, 1],
        "poison_ratio": 0.1,
        "target_class": 0,
    },
    "model_replacement": {
        "enabled": True,
        "type": "model_replacement",
        "malicious_clients": [0],
    },
}

# Defense configurations
DEFENSES = {
    "none": {"enabled": False},
    "krum": {
        "enabled": True,
        "type": "krum",
        "num_malicious": 2,
    },
    "trimmed_mean": {
        "enabled": True,
        "type": "trimmed_mean",
        "trim_ratio": 0.2,
    },
    "fltrust": {
        "enabled": True,
        "type": "fltrust",
    },
    "dp_sgd": {
        "enabled": True,
        "type": "dp_sgd",
        "clip_norm": 1.0,
        "noise_multiplier": 0.1,
    },
}

print(f"Seeds: {len(SEEDS)}")
print(f"Distributions: {len(DISTRIBUTIONS)}")
print(f"Attacks: {len(ATTACKS)}")
print(f"Defenses: {len(DEFENSES)}")

---
## Run Experiments

In [ ]:
# Create results directory
RESULTS_DIR = f"./experiments/mnist_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
os.makedirs(RESULTS_DIR, exist_ok=True)
print(f"Results will be saved to: {RESULTS_DIR}")

runner = ExperimentRunner(RESULTS_DIR)

In [ ]:
# Build experiment matrix
experiments = []

# Baseline (no attack, no defense)
for dist, alpha in DISTRIBUTIONS:
    experiments.append(("none", "none", dist, alpha))

# Full attack × defense matrix
for attack_name in ["label_flip", "backdoor", "model_replacement"]:
    for defense_name in DEFENSES.keys():
        for dist, alpha in DISTRIBUTIONS:
            experiments.append((attack_name, defense_name, dist, alpha))

total_runs = len(experiments) * len(SEEDS)
print(f"Total experiment configurations: {len(experiments)}")
print(f"Total runs (with seeds): {total_runs}")
print(f"Estimated time: {total_runs * 0.5:.0f} - {total_runs * 2:.0f} minutes")

In [ ]:
# Run all experiments
all_results = []
start_time = time.time()
completed = 0

for attack_name, defense_name, dist, alpha in experiments:
    seed_results = []
    
    for seed in SEEDS:
        completed += 1
        elapsed = time.time() - start_time
        eta = (elapsed / completed) * (total_runs - completed) if completed > 0 else 0
        
        print(f"[{completed}/{total_runs}] {attack_name}+{defense_name} | {dist}(α={alpha}) | seed={seed} | ETA: {eta/60:.1f}min")
        
        attack_cfg = ATTACKS[attack_name]
        defense_cfg = DEFENSES[defense_name]
        
        config = ExperimentConfig(
            name=f"mnist_{attack_name}_{defense_name}_{dist}_a{alpha}_s{seed}",
            dataset="mnist",
            num_clients=MNIST_CONFIG["num_clients"],
            num_rounds=MNIST_CONFIG["num_rounds"],
            local_epochs=MNIST_CONFIG["local_epochs"],
            batch_size=MNIST_CONFIG["batch_size"],
            learning_rate=MNIST_CONFIG["learning_rate"],
            partition=dist,
            seed=seed,
            attack_enabled=attack_cfg.get("enabled", False),
            attack_type=attack_cfg.get("type", "none"),
            malicious_clients=attack_cfg.get("malicious_clients", []),
            poison_ratio=attack_cfg.get("poison_ratio", 0.1),
            target_class=attack_cfg.get("target_class", 0),
            defense_enabled=defense_cfg.get("enabled", False),
            defense_type=defense_cfg.get("type", "none"),
            num_malicious_assumed=defense_cfg.get("num_malicious", 2),
            trim_ratio=defense_cfg.get("trim_ratio", 0.1),
        )
        
        try:
            result = runner.run_simulation(config)
            seed_results.append({
                "attack": attack_name,
                "defense": defense_name,
                "distribution": dist,
                "alpha": alpha,
                "seed": seed,
                "accuracy": result.final_accuracy,
                "loss": result.final_loss,
                "asr": result.attack_success_rate if result.attack_success_rate else 0.0
            })
        except Exception as e:
            print(f"  ERROR: {e}")
            continue
    
    # Aggregate results for this config
    if seed_results:
        accs = [r["accuracy"] for r in seed_results]
        aggregate = {
            "attack": attack_name,
            "defense": defense_name,
            "distribution": dist,
            "alpha": alpha,
            "mean_accuracy": np.mean(accs),
            "std_accuracy": np.std(accs, ddof=1) if len(accs) > 1 else 0.0,
            "num_seeds": len(seed_results),
            "raw_results": seed_results
        }
        all_results.append(aggregate)
        print(f"  → Accuracy: {aggregate['mean_accuracy']*100:.2f}% ± {aggregate['std_accuracy']*100:.2f}%")

total_time = time.time() - start_time
print(f"\n{'='*50}")
print(f"EXPERIMENTS COMPLETE")
print(f"Total time: {total_time/60:.1f} minutes")
print(f"{'='*50}")

---
## Save Results

In [ ]:
# Save JSON results
results_path = os.path.join(RESULTS_DIR, "mnist_results.json")
with open(results_path, 'w') as f:
    json.dump(all_results, f, indent=2)
print(f"Results saved to: {results_path}")

In [ ]:
# Generate LaTeX table
def generate_latex_table(results, distribution, alpha=None):
    """Generate LaTeX table for paper."""
    filtered = [r for r in results if r["distribution"] == distribution]
    if alpha is not None:
        filtered = [r for r in filtered if r["alpha"] == alpha]
    
    dist_label = "IID" if distribution == "iid" else f"Non-IID (α={alpha})"
    
    lines = [
        f"% Table: {dist_label} Results",
        "\\begin{table}[htbp]",
        "\\centering",
        f"\\caption{{MNIST Attack and Defense Performance under {dist_label} Distribution}}",
        f"\\label{{tab:mnist_{distribution}_results}}",
        "\\begin{tabular}{llc}",
        "\\toprule",
        "Attack & Defense & Accuracy (\\%) \\\\",
        "\\midrule",
    ]
    
    for r in filtered:
        acc_str = f"{r['mean_accuracy']*100:.2f} $\\pm$ {r['std_accuracy']*100:.2f}"
        attack_display = r['attack'].replace('_', ' ').title()
        defense_display = r['defense'].replace('_', ' ').title()
        lines.append(f"{attack_display} & {defense_display} & {acc_str} \\\\")
    
    lines.extend([
        "\\bottomrule",
        "\\end{tabular}",
        "\\end{table}",
    ])
    
    return "\n".join(lines)

# Generate and save LaTeX tables
latex_content = "% Auto-generated LaTeX tables for paper\n\n"
latex_content += generate_latex_table(all_results, "iid") + "\n\n"
latex_content += generate_latex_table(all_results, "noniid", 0.5) + "\n\n"
latex_content += generate_latex_table(all_results, "noniid", 0.1) + "\n\n"

latex_path = os.path.join(RESULTS_DIR, "mnist_tables.tex")
with open(latex_path, 'w') as f:
    f.write(latex_content)
print(f"LaTeX tables saved to: {latex_path}")

---
## Quick Summary

In [ ]:
import pandas as pd

# Create summary dataframe
summary_data = []
for r in all_results:
    dist_label = "IID" if r["distribution"] == "iid" else f"Non-IID (α={r['alpha']})"
    summary_data.append({
        "Distribution": dist_label,
        "Attack": r["attack"].replace("_", " ").title(),
        "Defense": r["defense"].replace("_", " ").title(),
        "Accuracy": f"{r['mean_accuracy']*100:.2f} ± {r['std_accuracy']*100:.2f}",
    })

df = pd.DataFrame(summary_data)
print("\nMNIST Experiment Results Summary:")
print(df.to_string(index=False))

---
## Next Steps

Results are saved! Proceed to:
- **05_cub200_experiments.ipynb** for CUB-200 experiments
- **06_results_analysis.ipynb** for visualization